In [1]:
import pandas as pd  #importing all the important packages
import numpy as np

In [2]:
class Pokemon(object): #inherit object class
    def __init__(self, name, level, ivs, evs, nature, item, status, ability, moves, df):
        self.name = name
        self.level = level
        self.typ1 = df[df.index==name].TYPE1.values
        self.typ2 = df[df.index==name].TYPE2.values
        self.base = [int(df[df.index==name].HP.values), int(df[df.index==name].ATK.values), int(df[df.index==name].DEF.values), int(df[df.index==name].SPATK.values), int(df[df.index==name].SPDEF.values), int(df[df.index==name].SPD.values)]
        self.total = sum(self.base)
        if checkIVs(ivs):
            self.IV = ivs
        else:
            raise Exception("Impossible IVs.")
        
        if checkEVs(evs):
            self.EV = evs
        else:
            raise Exception("Impossible EVs.")
        
        if checkItem(item):
            self.item = item
        else:
            raise Exception("Item doesn't exist.")
        
        if checkNature(nature):
            self.nature = nature
        else:
            raise Exception("Nature doesn't exist.")
        
        if checkStatus(status):
            self.status = status
        else:
            raise Exception("Status's are BRN, PSN, bad_PSN, PAR, FRZ, SLP or None. Provided status doesn't exist.")
            
        if (ability == df[df.index==name].ABILITY1.values) or (ability == df[df.index==name].ABILITY2.values) or (ability == df[df.index==name].ABILITY3.values):
            self.ability = ability
        else:
            raise Exception("Ability doesn't match pokemon")
        self.HP = self.getStats()[0]
        self.fainted = False
        self.moves = moves
    
    #defaults to no EVs, no IVs, no item, no status, neutral nature
    @classmethod
    def base(cls, name, level, ability, moves, df):
        return cls(name, level, [0,0,0,0,0,0], [0,0,0,0,0,0], 'Hardy', None, None, ability, moves, df)
    
    #defaults with item
    @classmethod
    def item(cls, name, level, ability, moves, df, item):
        return cls(name, level, [0,0,0,0,0,0], [0,0,0,0,0,0], 'Hardy', item, None, ability, moves, df)
    
    #defaults with nature
    @classmethod
    def nature(cls, name, level, ability, moves, df, nature):
        return cls(name, level, [0,0,0,0,0,0], [0,0,0,0,0,0], nature, None, None, ability, moves, df)
    
    #defaults with ivs and nature
    @classmethod
    def ivs(cls, name, level, ability, moves, df, nature, ivs):
        return cls(name, level, ivs, [0,0,0,0,0,0], nature, None, None, ability, moves, df)
    
    #defaults with evs, ivs and nature
    @classmethod
    def evs(cls, name, level, ability, moves, df, nature, ivs, evs):
        return cls(name, level, ivs, evs, nature, None, None, ability, moves, df)
    
    # instance method
    def show(self):
        print(self.name,'Level:', self.level,'\nBase:', self.base, 'Total:', self.total, '\nIVs:', self.IV, '\nEVs:', self.EV, '\nNature:', self.nature, 'Ability:', self.ability, '\nItem:', self.item,'Status:', self.status, '\nCurrent HP:', self.HP, '\nMoves: ', self.moves, '\nFainted :', self.fainted)
    
    #get methods
    def getName(self):
        return self.name
    def getLevel(self):
        return self.level
    def getTypes(self):
        return [self.typ1, self.typ2]
    def getBase(self):
        return self.base
    def getTotal(self):
        return self.total
    def getIV(self):
        return self.IV
    def getEV(self):
        return self.EV
    def getItem(self):
        return self.item
    def getNature(self):
        return self.nature
    def getStatus(self):
        return self.status
    def getAbility(self):
        return self.ability
    def getMoves(self):
        return self.moves
    
    #get stats
    def getStats(self):
        HP = ((2*self.base[0]+self.IV[0]+(self.EV[0]/4))*self.level)/100+self.level+10
        ATK = ((2*self.base[1]+self.IV[1]+(self.EV[1]/4))*self.level)/100+5
        DEF = ((2*self.base[2]+self.IV[2]+(self.EV[2]/4))*self.level)/100+5
        SPATK = ((2*self.base[3]+self.IV[3]+(self.EV[3]/4))*self.level)/100+5
        SPDEF = ((2*self.base[4]+self.IV[4]+(self.EV[4]/4))*self.level)/100+5
        SPD = ((2*self.base[5]+self.IV[5]+(self.EV[5]/4))*self.level)/100+5
        mat = natureMatrix(self.nature)
        return [int(HP*mat[0]),int(ATK*mat[1]),int(DEF*mat[2]),int(SPATK*mat[3]),int(SPDEF*mat[4]),int(SPD*mat[5])]
    
    #set methods
    def setLevel(self,level):
        self.level = level
    def setIV(self,ivs):
        self.IV = ivs
    def setEV(self,evs):
        self.EV = evs
    def setItem(self,item):
        self.item = item
    def setNature(self,nature):
        self.nature = nature
    def setStatus(self,status):
        self.status = status
    def setAbility(self,ability):
        self.ability = ability
    
    #function for healing HP
    def heal(self,heal):
        self.HP = self.HP + heal
        print(self.name + ' healed up to ' + str(heal) + ' points of damage!')
        if self.HP > self.getStats()[0]:
            self.HP = self.getStats()[0]
            print(self.name + ' is back to full health!')
        print('Current HP: ' + str(self.HP))
        return self.HP
    
    def dmg(self,dmg):
        self.HP = self.HP - dmg
        print(self.name + ' took up to ' + str(dmg) + ' points of damage!')
        if self.HP <= 0:
            if dmg >= self.getStats()[0] and item == 'Focus Sash':
                self.HP = 1
                self.item = None
                print(self.name + ' used its Focus Sash!')
            else:
                self.HP = 0
                self.fainted = True
                print(self.name + ' fainted!')
        print('Current HP: ' + str(self.HP))
        return self.HP
    

In [3]:
#check status is legitimate
def checkStatus(status):
    return (status == 'BRN') or (status == 'PSN') or (status == 'bad_PSN') or (status == 'PAR') or (status == 'FRZ') or (status == 'SLP') or (status == None)
#check item is legitimate
def checkItem(item):
    return True
#check ivs are legitimate
def checkIVs(ivs):
    i = 0
    while i < 6:
        if ivs[i] > 31 or ivs[i] < 0:
            return False
        i = i+1
    return True
#check evs are legitimate
def checkEVs(evs):
    i = 0
    while i < 6:
        if evs[i] > 252 or evs[i] < 0:
            return False
        i = i + 1
    if sum(evs) > 510:
        return False
    return True
#check nature is legitimate
def checkNature(nature):
    if nature == 'Lonely' or nature == 'Adamant' or nature == 'Brave' or nature == 'Naughty':
        return True
    if nature == 'Bold' or nature == 'Relaxed' or nature == 'Impish' or nature == 'Lax':
        return True
    if nature == 'Timid' or nature == 'Hasty' or nature == 'Jolly' or nature == 'Naive':
        return True
    if nature == 'Modest' or nature == 'Mild' or nature == 'Quiet' or nature == 'Rash':
        return True
    if nature == 'Calm' or nature == 'Gentle' or nature == 'Sassy' or nature == 'Careful':
        return True
    if nature == 'Hardy' or nature == 'Docile' or nature == 'Serious' or nature == 'Bashful' or nature == 'Quirky':
        return True
    return False

# return a matrix of stats with nature applied
def natureMatrix(nature):
    if checkNature(nature):
        mat = [1,1,1,1,1,1]
        if nature == 'Lonely' or nature == 'Adamant' or nature == 'Brave' or nature == 'Naughty':
            mat[1] = 1.1
        if nature == 'Bold' or nature == 'Relaxed' or nature == 'Impish' or nature == 'Lax':
            mat[2] = 1.1
        if nature == 'Timid' or nature == 'Hasty' or nature == 'Jolly' or nature == 'Naive':
            mat[3] = 1.1
        if nature == 'Modest' or nature == 'Mild' or nature == 'Quiet' or nature == 'Rash':
            mat[4] = 1.1
        if nature == 'Calm' or nature == 'Gentle' or nature == 'Sassy' or nature == 'Careful':
            mat[5] = 1.1
        if nature == 'Bold' or nature == 'Timid' or nature == 'Modest' or nature == 'Calm':
            mat[1] = 0.9
        if nature == 'Lonely' or nature == 'Hasty' or nature == 'Mild' or nature == 'Gentle':
            mat[2] = 0.9
        if nature == 'Adamant' or nature == 'Impish' or nature == 'Jolly' or nature == 'Careful':
            mat[3] = 0.9
        if nature == 'Naughty' or nature == 'Lax' or nature == 'Naive' or nature == 'Rash':
            mat[4] = 0.9
        if nature == 'Brave' or nature == 'Relaxed' or nature == 'Quiet' or nature == 'Sassy':
            mat[5] = 0.9
    else:
        raise Exception("Nature doesn't exist!")
    return mat

# Create Example Pokemon Objects

In [4]:
import pickle
pickle_in = open('../pokemon.pickle','rb')
df = pickle.load(pickle_in)
#change into upper case
df.columns = df.columns.str.upper().str.replace('_', '')
#some values in TYPE2 are so they have to be filled or deleted
df['TYPE2'] = df['TYPE2'].replace('None', np.nan)
df['TYPE2'].fillna(df['TYPE1'], inplace=True)

In [5]:
swampert = Pokemon.base("Swampert",40,"Torrent",[],df)
swampert.show()
print(swampert.getStats())
swampert.dmg(20)
swampert.heal(30)
swampert.dmg(80)
swampert.heal(25)

Swampert Level: 40 
Base: [100, 110, 90, 85, 90, 60] Total: 535 
IVs: [0, 0, 0, 0, 0, 0] 
EVs: [0, 0, 0, 0, 0, 0] 
Nature: Hardy Ability: Torrent 
Item: None Status: None 
Current HP: 130 
Moves:  [] 
Fainted : False
[130, 93, 77, 73, 77, 53]
Swampert took up to 20 points of damage!
Current HP: 110
Swampert healed up to 30 points of damage!
Swampert is back to full health!
Current HP: 130
Swampert took up to 80 points of damage!
Current HP: 50
Swampert healed up to 25 points of damage!
Current HP: 75


75

In [6]:
bulbasaur = Pokemon.ivs("Bulbasaur",10,"Overgrow",[],df,'Bold',[31,31,31,31,31,31])
bulbasaur.show()
bulbasaur.getStats()

Bulbasaur Level: 10 
Base: [45, 49, 49, 65, 65, 45] Total: 318 
IVs: [31, 31, 31, 31, 31, 31] 
EVs: [0, 0, 0, 0, 0, 0] 
Nature: Bold Ability: Overgrow 
Item: None Status: None 
Current HP: 32 
Moves:  [] 
Fainted : False


[32, 16, 19, 21, 21, 17]

In [7]:
pignite = Pokemon.evs("Pignite",26,"Blaze",[],df,'Bold',[31,31,31,31,31,31],[0,252,0,0,252,4])
pignite.show()
pignite.getStats()

Pignite Level: 26 
Base: [90, 93, 55, 70, 55, 55] Total: 418 
IVs: [31, 31, 31, 31, 31, 31] 
EVs: [0, 252, 0, 0, 252, 4] 
Nature: Bold Ability: Blaze 
Item: None Status: None 
Current HP: 90 
Moves:  [] 
Fainted : False


[90, 70, 45, 49, 58, 41]